In [ ]:
! pip install wandb

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
import numpy as np
import cv2
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.login()

In [4]:
wandb.init(project="Animals_Recognition", entity="gharabadiyan")

wandb: Currently logged in as: gharabadiyan. Use `wandb login --relogin` to force relogin


In [5]:
dataset_path = '/content/drive/MyDrive/animal'
width = height = 224
batch_size = 32


idg = ImageDataGenerator(

    rescale = 1./255,
    horizontal_flip = True,
    #zoom_range = 0.1,
    brightness_range = (0.9,1.1),
    #rotation_range = 15,
    validation_split = 0.2 #validation data
)

train_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "training"
)

val_data = idg.flow_from_directory(  #eamal taghirat bar roye tasavir
    dataset_path,
    target_size = (width,height),
    class_mode = "categorical",
    batch_size = batch_size,
    subset = "validation"
)

Found 1042 images belonging to 5 classes.
Found 257 images belonging to 5 classes.


In [6]:
base_model = tf.keras.applications.MobileNetV2(
    include_top=False,
    weights = 'imagenet',
    input_shape=(width,height,3),
    pooling = 'avg',
)

model = tf.keras.models.Sequential([
        base_model,
        Dropout(0.3),
        # Flatten(),
        # Dense(1024,activation='relu'),
        Dense(256,activation='relu'),
        Dense(5,activation='softmax')

])

9406464/9406464 [==============================] - 1s 0us/step


In [8]:
for layer in base_model.layers[:-3]:  #freeze layer
  layer.trainable = False

In [9]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [10]:
model.fit(train_data,validation_data=val_data,epochs=10,callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
33/33 [==============================] - ETA: 0s - loss: 0.4366 - accuracy: 0.8464

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231028_085305-ve2sd4bt/files/model-best)... Done. 0.1s


33/33 [==============================] - 434s 13s/step - loss: 0.4366 - accuracy: 0.8464 - val_loss: 0.0756 - val_accuracy: 0.9805
Epoch 2/10
33/33 [==============================] - ETA: 0s - loss: 0.1045 - accuracy: 0.9616

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231028_085305-ve2sd4bt/files/model-best)... Done. 0.1s


...

33/33 [==============================] - 36s 1s/step - loss: 0.0347 - accuracy: 0.9923 - val_loss: 0.0511 - val_accuracy: 0.9767
Epoch 6/10
33/33 [==============================] - ETA: 0s - loss: 0.0412 - accuracy: 0.9837

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231028_085305-ve2sd4bt/files/model-best)... Done. 0.1s


33/33 [==============================] - 36s 1s/step - loss: 0.0412 - accuracy: 0.9837 - val_loss: 0.0490 - val_accuracy: 0.9805
Epoch 7/10
33/33 [==============================] - 21s 618ms/step - loss: 0.0224 - accuracy: 0.9933 - val_loss: 0.0836 - val_accuracy: 0.9689
Epoch 8/10
33/33 [==============================] - 19s 580ms/step - loss: 0.0315 - accuracy: 0.9875 - val_loss: 0.0608 - val_accuracy: 0.9805
Epoch 9/10
33/33 [==============================] - 21s 625ms/step - loss: 0.0232 - accuracy: 0.9914 - val_loss: 0.0558 - val_accuracy: 0.9767
Epoch 10/10
33/33 [==============================] - ETA: 0s - loss: 0.0183 - accuracy: 0.9914

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231028_085305-ve2sd4bt/files/model-best)... Done. 0.1s


33/33 [==============================] - 35s 1s/step - loss: 0.0183 - accuracy: 0.9914 - val_loss: 0.0427 - val_accuracy: 0.9922
